In [4]:
from tbats import TBATS, BATS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

In [6]:
## function to fomat the data
def data_format(df):
    df=df.drop(['Unnamed: 0','campaign_name', 'adset_name', 'ad_name','inline_link_click_ctr', 'inline_post_engagement',
       'instant_experience_clicks_to_open',
       'instant_experience_clicks_to_start',
       'instant_experience_outbound_clicks', 'interactive_component_tap_value',
       'objective','updated_time','website_ctr_link_click'],axis=1)
    df=df.rename(columns={'cost_per_action_type_video_view':'cpa_video_view'})
    df=df.rename(columns={'cost_per_action_type_link_click':'cpa_link_click'})
    df=df.rename(columns={'cost_per_action_type_onsite_conversion.post_save':'cpa_on_conv'})
    df=df.rename(columns={'cost_per_action_type_omni_activate_app':'cpa_all_act_app'})
    df=df.rename(columns={'cost_per_action_type_post_engagement':'cpa_post_eng'})
    df=df.rename(columns={'cost_per_action_type_page_engagement':'cpa_pg_eng'})
    df=df.rename(columns={'cost_per_action_type_omni_app_install':'cpa_all_app_install'})
    df=df.rename(columns={'cost_per_inline_link_click':'cpinl_link_click'})
    df=df.rename(columns={'cost_per_inline_post_engagement':'cpinl_post_eng'})
    df=df.rename(columns={'cost_per_outbound_click_outbound_click':'cpo_click'})
    df=df.rename(columns={'hourly_stats_aggregated_by_advertiser_time_zone':'time'})
    df=df.rename(columns={'actions_mobile_app_install':'mobile_conv'})
    df=df.rename(columns={'actions_omni_app_install':'all_conv'})
    df['CR']=round((df.mobile_conv/df.clicks) *100, 2)
    df['CTR']=round((df.clicks / df.impressions) * 100,4)
    df['CPC']= round(df.spend / df.clicks,2)
    df['costPerCon'] = round(df.spend / df.mobile_conv, 2)
    df['CPM'] = round((df.spend / df.impressions) * 1000, 2)

    df=df.dropna(subset=['time'])
    df.time=df.time.replace('00:00:00 - 00:59:59',1)
    df.time=df.time.replace('01:00:00 - 01:59:59',2)
    df.time=df.time.replace('02:00:00 - 02:59:59',3)
    df.time=df.time.replace('03:00:00 - 03:59:59',4)
    df.time=df.time.replace('04:00:00 - 04:59:59',5)
    df.time=df.time.replace('05:00:00 - 05:59:59',6)
    df.time=df.time.replace('06:00:00 - 06:59:59',7)
    df.time=df.time.replace('07:00:00 - 07:59:59',8)
    df.time=df.time.replace('08:00:00 - 08:59:59',9)
    df.time=df.time.replace('09:00:00 - 09:59:59',10)
    df.time=df.time.replace('10:00:00 - 10:59:59',11)
    df.time=df.time.replace('11:00:00 - 11:59:59',12)
    df.time=df.time.replace('12:00:00 - 12:59:59',13)
    df.time=df.time.replace('13:00:00 - 13:59:59',14)
    df.time=df.time.replace('14:00:00 - 14:59:59',15)
    df.time=df.time.replace('15:00:00 - 15:59:59',16)
    df.time=df.time.replace('16:00:00 - 16:59:59',17)
    df.time=df.time.replace('17:00:00 - 17:59:59',18)
    df.time=df.time.replace('18:00:00 - 18:59:59',19)
    df.time=df.time.replace('19:00:00 - 19:59:59',20)
    df.time=df.time.replace('20:00:00 - 20:59:59',21)
    df.time=df.time.replace('21:00:00 - 21:59:59',22)
    df.time=df.time.replace('22:00:00 - 22:59:59',23)
    df.time=df.time.replace('23:00:00 - 23:59:59',24)
    df['day']=pd.to_datetime(df['date_start']).dt.day
    df['Time']=df['time']#+24*(df['day']-1)
    df['week']=pd.to_datetime(df['date_start']).dt.week
    df.drop(df.tail(1).index,inplace = True)
    return df

file='../Timing Analysis/data/ad_Aug_hourly_advertiser_time.csv'
df=pd.read_csv(file)
df=data_format(df)

In [7]:
grouped = pd.DataFrame(df.groupby(['Time','week'])['mobile_conv'].sum().reset_index())

In [9]:
train = grouped.mobile_conv[:80]
test = grouped.mobile_conv[80:]

In [18]:
estimator = TBATS(seasonal_periods=(1/24, 7))
model = estimator.fit(train)
# # Forecast 365 days ahead
y_forecast = model.forecast(steps=4)

/opt/conda/lib/python3.7/site-packages/tbats/error/ExceptionHandler.py:13: InputArgsWarning: All seasonal periods should be values greater than 1. Ignoring all seasonal period values that do not meet this condition.
  warnings.warn(message, warning_type)


In [19]:
y_forecast

array([ 49.33363392,  38.8703064 , -14.34213724,   6.09668576])

In [13]:
grouped.tail(64)

,Time,week,mobile_conv
80,14,33,62.0
81,14,34,26.0
82,14,35,6.0
83,14,36,1.0
84,15,31,40.0
...,...,...,...
139,24,32,128.0
140,24,33,110.0
141,24,34,82.0
142,24,35,30.0
